In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

### Creating Training Data Structure

In [2]:
df = pd.read_csv("/Users/anurag/Desktop/Computer Science/Monash/2023/FIT3164 Data Science Project Semester 2/Electricity-Demand-Forecasting/Data/data_for_analysis/actuals_1.csv")
df2 = pd.read_csv("/Users/anurag/Desktop/Computer Science/Monash/2023/FIT3164 Data Science Project Semester 2/Electricity-Demand-Forecasting/Data/data_for_analysis/actuals_2.csv")

#Create Whole Dataset
df_train = pd.concat([df, df2], axis = 0)

df_train['Time'] = pd.to_datetime(df_train['Time'])

/var/folders/05/dgx_ky6d3n9f_vxn26lcln1h0000gn/T/ipykernel_71295/2458164464.py:7: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_train['Time'] = pd.to_datetime(df_train['Time'])


### Creating Test Data Structure

In [3]:
df_test = pd.read_csv("/Users/anurag/Desktop/Computer Science/Monash/2023/FIT3164 Data Science Project Semester 2/Electricity-Demand-Forecasting/Data/Forecast_Data/forecasts.csv")

df_test['Time'] = pd.to_datetime(df_test['Time'])

/var/folders/05/dgx_ky6d3n9f_vxn26lcln1h0000gn/T/ipykernel_71295/2712329862.py:3: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_test['Time'] = pd.to_datetime(df_test['Time'])


In [4]:
def combine_rows_with_different_attribute_names(df, df_test):
    combined_data_list = []

    # start at 0, stop 168 + 1 rows before end, jump by 24 hours / 1 day.
    for i in range(0, len(df_test) - (168 + 1), 24):
        combined_data = {}
        
        if i + 168 + 24 >= (len(df_test)):
            break
        
        for j in range(168):
            row = df.iloc[i + j]

            # Iterate over columns and update column names
            for col, val in row.items():
                combined_data[f"{col}_date{j+1}"] = val
        
        # Append the combined data dictionary to the list
        
        for j in range(48):
            # if i + j + 168 >= (len(df_test) - 1):
            #     break
            row = df_test.iloc[i + j + 168]
            
            for col, val in row.items():
                combined_data[f"{col}_test_date{j+1}"] = val
            
            
        combined_data_list.append(combined_data)

    # Convert the list of dictionaries into a DataFrame
    combined_df = pd.DataFrame(combined_data_list)

    return combined_df

# Example usage:
# Assuming you have a DataFrame named df_train
formatted_data = combine_rows_with_different_attribute_names(df_train, df_test)


In [36]:
def naive(formatted_data):
    """
    :Input: 
        :data: The entire dataset of batches.
        
    For this algorithm, go over each batch, create a list of predctions,
    compare these predictions to the predictions to actuals, which can be done through the 
    next row which introduce the data for the next two days.
    """

    predictions = []
    actuals = []
    for index, row in formatted_data.iterrows():
        predictions.append([])
        actuals.append([])
        for i in range(121, 169):
            column_load = f"Load (kW)_date{i}"
            column_time = f"Time_date{i}"
            
            # predictions[index] = [load, time of load used to make prediction]
            # The load above is for the time to the right plus two days.
            
            # predictions[index].append([row[column_load], row[column_time]])
            
            # Just Predicted Load
            predictions[index].append(row[column_load]) 

            
            # Append the same data into actuals, but with a two-row offset
            offset_index = index + 2
            if offset_index < len(formatted_data):
                actual_row = formatted_data.iloc[offset_index]
                # actuals[index].append([actual_row[column_load], actual_row[column_time]])
                
                # Just Actual Load
                actuals[index].append(actual_row[column_load])
                
            else:
                continue

    return predictions, actuals

    
x, y = naive(formatted_data)


In [49]:
# Initialize empty lists to store the metrics
mae_per_sublist = []
rmse_per_sublist = []
mape_per_sublist = []
r2_per_sublist = []

for i in range(len(x)):
    if len(x[i]) == 0 or len(y[i]) == 0:
        break
    
    predicted_load = np.array(x[i])
    actual_load = np.array(y[i])
    
    # Calculate the MAE for the current sublist
    mae = mean_absolute_error(actual_load, predicted_load)
    
    # Calculate the RMSE for the current sublist
    mse = mean_squared_error(actual_load, predicted_load)
    rmse = np.sqrt(mse)
    
    # Calculate the MAPE for the current sublist
    mape = np.mean(np.abs((actual_load - predicted_load) / actual_load)) * 100
    
    # Calculate the R-squared (R²) for the current sublist
    r_squared = r2_score(actual_load, predicted_load)
    
    # Append Metrics
    mae_per_sublist.append(mae)
    rmse_per_sublist.append(rmse)
    mape_per_sublist.append(mape)
    r2_per_sublist.append(r_squared)

# Check if there are sublists with non-empty data
if len(mae_per_sublist) > 0:
    # Calculate the overall metrics by taking the mean of all sublists' values
    overall_mae = np.mean(mae_per_sublist)
    overall_rmse = np.mean(rmse_per_sublist)
    overall_mape = np.mean(mape_per_sublist)
    overall_r2 = np.mean(r2_per_sublist)

    # Print the metrics for each sublist and the overall metrics
    # for i in range(len(mae_per_sublist)):
    #     print(f"Metrics for sublist {i+1}:")
    #     print(f"MAE: {mae_per_sublist[i]}")
    #     print(f"RMSE: {rmse_per_sublist[i]}")
    #     print(f"MAPE: {mape_per_sublist[i]}")
    #     print(f"R-squared (R²): {r2_per_sublist[i]}\n")

    print("Overall Metrics:")
    print(f"Overall MAE: {overall_mae}")
    print(f"Overall RMSE: {overall_rmse}")
    print(f"Overall MAPE: {overall_mape}")
    print(f"Overall R-squared (R²): {overall_r2}")
else:
    print("No valid data to calculate metrics.")


Overall Metrics:
Overall MAE: 0.04561668893653571
Overall RMSE: 0.0612932683218121
Overall MAPE: 6.194156052274302
Overall R-squared (R²): -0.19332274430508656
